In [1]:
!nvidia-smi

Sun Aug  1 02:32:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Directory 설정, 구글 드라이브 import

In [2]:
cur_dir = '/content/drive/MyDrive/KLUE_TC'

## Utils

In [3]:
!pip install adamp
!pip install git+https://github.com/GY-Jeong/transformers

  Cloning https://github.com/GY-Jeong/transformers to /tmp/pip-req-build-lv5r9mrz
  Running command git clone -q https://github.com/GY-Jeong/transformers /tmp/pip-req-build-lv5r9mrz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [4]:
import os
import random
import torch
import numpy as np
from torch import nn

from torch.optim import Adam, AdamW, SGD
from adamp import AdamP
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR, ExponentialLR, CosineAnnealingWarmRestarts

from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup


def set_seeds(seed=42):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.benchmark = False


def save_checkpoint(state, model_dir, model_filename):
    print('saving model ...')
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(state, os.path.join(model_dir, model_filename))
    # torch.save(state, os.path.join(model_filename))


def get_optimizer(model, args):
    if args.optimizer == 'adam':
        optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    elif args.optimizer == 'adamW':
        optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    elif args.optimizer == 'adamP':
        optimizer = AdamP(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    elif args.optimizer == 'SGD':
        optimizer = SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    # 모든 parameter들의 grad값을 0으로 초기화
    optimizer.zero_grad()

    return optimizer


def get_scheduler(optimizer, args):
    if args.scheduler == 'plateau':
        scheduler = ReduceLROnPlateau(optimizer, patience=args.plateau_patience, factor=args.plateau_factor, mode='max',
                                      verbose=True)
    elif args.scheduler == 'linear_warmup':
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                    num_training_steps=args.total_steps)
    elif args.scheduler == 'step_lr':
        scheduler = StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
    elif args.scheduler == 'exp_lr':
        scheduler = ExponentialLR(optimizer, gamma=args.gamma)
    elif args.scheduler == 'cosine_annealing':
        scheduler = CosineAnnealingLR(optimizer, T_max=args.t_max, eta_min=args.eta_min)
    elif args.scheduler == 'cosine_annealing_warmstart':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=args.T_0, T_mult=args.T_mult, eta_min=args.eta_min,
                                                last_epoch=-1)

    return scheduler


def update_params(loss, model, optimizer, batch_idx, max_len, args):
    if args.gradient_accumulation:
        # normalize loss to account for batch accumulation
        loss = loss / args.accum_iter 

        # backward pass
        loss.backward()

        # weights update
        if ((batch_idx + 1) % args.accum_iter == 0) or (batch_idx + 1 == max_len):
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad)
            optimizer.step()
            optimizer.zero_grad()
    else:
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_grad)
        optimizer.step()
        optimizer.zero_grad()


def load_tokenizer(args):
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name
        if args.tokenizer_name
        else args.model_name_or_path,
        use_fast=True,
    )

    return tokenizer


def load_model(args, model_name=None):
    if not model_name:
        model_name = args.model_name
    model_path = os.path.join(args.model_dir, model_name)
    print("Loading Model from:", model_path)
    load_state = torch.load(model_path)
    # load_state = torch.load(model_name)

    # Load pretrained model and tokenizer
    config = AutoConfig.from_pretrained(
        args.config_name
        if args.config_name
        else args.model_name_or_path,
    )

    config.num_labels = 7

    model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    ).to(args.device)

    # model.classifier = nn.Sequential(
    #     nn.Dropout(p=0.3, inplace=False),
    #     nn.Linear(1024, 1024),
    #     nn.Tanh(),
    #     nn.Dropout(p=0.3, inplace=False),
    #     nn.Linear(1024, 512),
    #     nn.Tanh(),
    #     nn.Dropout(p=0.3, inplace=False),
    #     nn.Linear(512, 7),
    # )

    model.load_state_dict(load_state['state_dict'], strict=True)

    # model = model.to(args.device)

    print("Loading Model from:", model_path, "...Finished.")

    return model


def get_model(args):
    # Load pretrained model and tokenizer
    config = AutoConfig.from_pretrained(
        args.config_name
        if args.config_name
        else args.model_name_or_path,
    )

    config.num_labels = 7
    model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    ).to(args.device)

    # model.classifier = nn.Sequential(
    #     nn.Dropout(p=0.3, inplace=False),
    #     nn.Linear(1024, 1024),
    #     nn.Tanh(),
    #     nn.Dropout(p=0.3, inplace=False),
    #     nn.Linear(1024, 512),
    #     nn.Tanh(),
    #     nn.Dropout(p=0.3, inplace=False),
    #     nn.Linear(512, 7),
    # )

    # print(model)
    #model.classifier.dropout = nn.Dropout(p=0.3, inplace = False)

    model = model.to(args.device)

    return model


def get_loaders(args, train, valid, is_inference=False):
    pin_memory = True
    train_loader, valid_loader = None, None

    if is_inference:
        test_dataset = YNAT_dataset(args, valid, is_inference)
        test_loader = torch.utils.data.DataLoader(test_dataset, num_workers=args.num_workers, shuffle=False,
                                                  batch_size=args.batch_size, pin_memory=pin_memory)
        return test_loader

    if train is not None:
        train_dataset = YNAT_dataset(args, train, is_inference)
        train_loader = torch.utils.data.DataLoader(train_dataset, num_workers=args.num_workers, shuffle=True,
                                                   batch_size=args.batch_size, pin_memory=pin_memory)
    if valid is not None:
        valid_dataset = YNAT_dataset(args, valid, is_inference)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, num_workers=args.num_workers, shuffle=False,
                                                   batch_size=args.batch_size, pin_memory=pin_memory)

    return train_loader, valid_loader


# loss계산하고 parameter update!
def compute_loss(preds, targets, args):
    """
    Args :
        preds   : (batch_size, max_seq_len)
        targets : (batch_size, max_seq_len)
    """
    # print(preds, targets)
    loss = get_criterion(preds, targets, args)
    # 마지막 시퀀스에 대한 값만 loss 계산
    # loss = loss[:, -1]
    # loss = torch.mean(loss)
    return loss


def get_criterion(pred, target, args):
    if args.criterion == 'BCE':
        loss = nn.BCELoss(reduction="none")
    elif args.criterion == "BCELogit":
        loss = nn.BCEWithLogitsLoss(reduction="none")
    elif args.criterion == "MSE":
        loss = nn.MSELoss(reduction="none")
    elif args.criterion == "L1":
        loss = nn.L1Loss(reduction="none")
    elif args.criterion == "CE":
        #weights = [1,1,2,1,1,1,1] #as class distribution
        #class_weights = torch.FloatTensor(weights).cuda()
        #loss = nn.CrossEntropyLoss(weight=class_weights)
        loss = nn.CrossEntropyLoss()
    # NLL, CrossEntropy not available
    return loss(pred, target)


def make_vocab(args):
    print("============ READ VOCABS ============")
    vocabs = []
    for i in range(7):
        vocab = set()
        f = open(args.vocab_dir + str(i) + '.txt', 'r')
        while True:
            line = f.readline()
            if not line: break
            vocab.add(line[:-1])
        f.close()
        vocabs.append(vocab)
        print(f"category {i} reading end, size : {len(vocab)}")
    return vocabs

## Dataloader

In [5]:
import os
import torch
import pandas as pd


class Preprocess:
    def __init__(self, args):
        self.args = args
        self.train_data = None
        self.test_data = None

    def load_data(self, file_name):
        csv_file_name = os.path.join(self.args.data_dir, file_name)
        df = pd.read_csv(csv_file_name)
        #del df['Unnamed: 0']
        return df.values

    def load_train_data(self):
        self.train_data = self.load_data('train_data.csv')

    def load_test_data(self):
        self.test_data = self.load_data('test_data.csv')


class YNAT_dataset(torch.utils.data.Dataset):
    def __init__(self, args, data, is_inference):
        self.args = args
        self.data = data
        self.is_inference = is_inference

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data[index]
        element = [row[i] for i in range(len(row))]
        #print(type(row))
        # np.array -> torch.tensor 형변환
        #for i, col in enumerate(row):
        #    if type(col) == str:
        #        pass
        #    else:
        #        row[i] = torch.tensor(col)

        return element



## Trainer

In [6]:
from sklearn.metrics import accuracy_score
from torch.nn.functional import one_hot
from tqdm import tqdm
from sklearn import metrics


def run(args, tokenizer, train_data, valid_data, cv_count):
    train_loader, valid_loader = get_loaders(args, train_data, valid_data)

    # only when using warmup scheduler
    # args.total_steps = int(len(train_loader.dataset) / args.batch_size) * args.n_epochs
    # args.warmup_steps = int(args.total_steps * args.warmup_ratio)

    model = get_model(args)
    optimizer = get_optimizer(model, args)
    scheduler = get_scheduler(optimizer, args)

    best_acc = -1
    early_stopping_counter = 0
    for epoch in range(args.n_epochs):

        print(f"Start Training: Epoch {epoch + 1}")

        if not args.cv_strategy:
            model_name = args.run_name
        else:
            model_name = f"{args.run_name.split('.pt')[0]}_{cv_count}.pt"

        # TRAIN
        train_acc, train_loss = train(args, model, tokenizer, train_loader, optimizer)

        # VALID
        acc, val_loss = validate(args, model, tokenizer, valid_loader)

        # TODO: model save or early stopping
        if args.scheduler == 'plateau':
            last_lr = optimizer.param_groups[0]['lr']
        else:
            last_lr = scheduler.get_last_lr()[0]

        print({"epoch": epoch, "train_loss": train_loss, "train_acc": train_acc,
                   "valid_acc": acc, "val_loss": val_loss, "learning_rate": last_lr})

        if acc > best_acc:
            best_acc = acc
            # torch.nn.DataParallel로 감싸진 경우 원래의 model을 가져옵니다.
            model_to_save = model.module if hasattr(model, 'module') else model
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model_to_save.state_dict(),
            },
                args.model_dir, model_name,
            )
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= args.patience:
                print(f'EarlyStopping counter: {early_stopping_counter} out of {args.patience}')
                break

        # scheduler
        if args.scheduler == 'plateau':
            scheduler.step(best_acc)
        else:
            scheduler.step()

    return best_acc


def inference(args, test_data):
    # ckpt_file_names = []
    all_fold_preds = []
    all_fold_argmax_preds = []

    if not args.cv_strategy:
        ckpt_file_names = [args.model_name]
    else:
        ckpt_file_names = [f"{args.model_name.split('.pt')[0]}_{i + 1}.pt" for i in range(args.fold_num)]

    tokenizer = load_tokenizer(args)

    for fold_idx, ckpt in enumerate(ckpt_file_names):
        model = load_model(args, ckpt)
        model.eval()
        test_loader = get_loaders(args, None, test_data, True)

        total_preds = []
        total_argmax_preds = []
        total_ids = []

        for step, batch in tqdm(enumerate(test_loader), desc='Inferencing', total=len(test_loader)):
            idx, text = batch
            tokenized_examples = tokenizer(
                text,
                max_length=args.max_seq_len,
                padding="max_length",
                return_tensors="pt"
            ).to(args.device)

            token_label_0_type_ids = []
            token_label_1_type_ids = []
            token_label_2_type_ids = []
            token_label_3_type_ids = []
            token_label_4_type_ids = []
            token_label_5_type_ids = []
            token_label_6_type_ids = []

            for row in tokenized_examples['input_ids']:
                row = tokenizer.convert_ids_to_tokens(row)
                # print(row)
                label_0 = [1 if token in args.vocab[0] else 0 for token in row]
                label_1 = [1 if token in args.vocab[1] else 0 for token in row]
                label_2 = [1 if token in args.vocab[2] else 0 for token in row]
                label_3 = [1 if token in args.vocab[3] else 0 for token in row]
                label_4 = [1 if token in args.vocab[4] else 0 for token in row]
                label_5 = [1 if token in args.vocab[5] else 0 for token in row]
                label_6 = [1 if token in args.vocab[6] else 0 for token in row]
                # print(label_2)
                token_label_0_type_ids.append(label_0)
                token_label_1_type_ids.append(label_1)
                token_label_2_type_ids.append(label_2)
                token_label_3_type_ids.append(label_3)
                token_label_4_type_ids.append(label_4)
                token_label_5_type_ids.append(label_5)
                token_label_6_type_ids.append(label_6)
                # temp = []
                # row = tokenizer.convert_ids_to_tokens(row)
                # for token in row:
                #     for i, element in enumerate(args.vocab):
                #         if token in element:
                #             temp.append(i+1)
                #             break
                #     else:
                #         temp.append(0)
                # #print(temp)
                # token_label_type_ids.append(temp)

            # token_label_type_ids = torch.tensor(token_label_type_ids).to(args.device)
            # tokenized_examples['token_label_type_ids'] = token_label_type_ids

            token_label_0_type_ids = torch.tensor(token_label_0_type_ids).to(args.device)
            token_label_1_type_ids = torch.tensor(token_label_1_type_ids).to(args.device)
            token_label_2_type_ids = torch.tensor(token_label_2_type_ids).to(args.device)
            token_label_3_type_ids = torch.tensor(token_label_3_type_ids).to(args.device)
            token_label_4_type_ids = torch.tensor(token_label_4_type_ids).to(args.device)
            token_label_5_type_ids = torch.tensor(token_label_5_type_ids).to(args.device)
            token_label_6_type_ids = torch.tensor(token_label_6_type_ids).to(args.device)

            tokenized_examples['token_label_0_type_ids'] = token_label_0_type_ids
            tokenized_examples['token_label_1_type_ids'] = token_label_1_type_ids
            tokenized_examples['token_label_2_type_ids'] = token_label_2_type_ids
            tokenized_examples['token_label_3_type_ids'] = token_label_3_type_ids
            tokenized_examples['token_label_4_type_ids'] = token_label_4_type_ids
            tokenized_examples['token_label_5_type_ids'] = token_label_5_type_ids
            tokenized_examples['token_label_6_type_ids'] = token_label_6_type_ids

            # token_label_type_ids = []
            # for row in tokenized_examples['input_ids']:
            #     temp = []
            #     row = tokenizer.convert_ids_to_tokens(row)
            #     for token in row:
            #         for i, element in enumerate(args.vocab):
            #             if token in element:
            #                 temp.append(i+1)
            #                 break
            #         else:
            #             temp.append(0)
            #     #print(temp)
            #     token_label_type_ids.append(temp)

            # token_label_type_ids = torch.tensor(token_label_type_ids).to(args.device)
            # tokenized_examples['token_label_type_ids'] = token_label_type_ids

            preds = model(**tokenized_examples)
            logits = preds['logits']
            #logits = logits[:,0,:]
            argmax_logits = torch.argmax(logits, dim=1)

            if args.device == 'cuda':
                logits = logits.to('cpu').detach().numpy()
                argmax_logits = argmax_logits.to('cpu').detach().numpy()
                # token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
                token_label_0_type_ids = token_label_0_type_ids.to('cpu').detach().numpy()
                token_label_1_type_ids = token_label_1_type_ids.to('cpu').detach().numpy()
                token_label_2_type_ids = token_label_2_type_ids.to('cpu').detach().numpy()
                token_label_3_type_ids = token_label_3_type_ids.to('cpu').detach().numpy()
                token_label_4_type_ids = token_label_4_type_ids.to('cpu').detach().numpy()
                token_label_5_type_ids = token_label_5_type_ids.to('cpu').detach().numpy()
                token_label_6_type_ids = token_label_6_type_ids.to('cpu').detach().numpy()
            else:  # cpu
                logits = logits.detach().numpy()
                argmax_logits = argmax_logits.detach().numpy()
                # token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
                token_label_0_type_ids = token_label_0_type_ids.detach().numpy()
                token_label_1_type_ids = token_label_1_type_ids.detach().numpy()
                token_label_2_type_ids = token_label_2_type_ids.detach().numpy()
                token_label_3_type_ids = token_label_3_type_ids.detach().numpy()
                token_label_4_type_ids = token_label_4_type_ids.detach().numpy()
                token_label_5_type_ids = token_label_5_type_ids.detach().numpy()
                token_label_6_type_ids = token_label_6_type_ids.detach().numpy()

            total_preds += list(logits)
            total_argmax_preds += list(argmax_logits)
            total_ids += list(idx)

        all_fold_preds.append(total_preds)
        all_fold_argmax_preds.append(total_argmax_preds)

        output_file_name = "output.csv" if not args.cv_strategy else f"output_{fold_idx + 1}.csv"
        write_path = os.path.join(args.output_dir, output_file_name)
        if not os.path.exists(args.output_dir):
            os.makedirs(args.output_dir)
        with open(write_path, 'w', encoding='utf8') as w:
            print("writing prediction : {}".format(write_path))
            w.write("index,topic_idx\n")
            for index, p in zip(total_ids, total_argmax_preds):
                w.write('{},{}\n'.format(index, p))

    if len(all_fold_preds) > 1:
        # Soft voting ensemble
        votes = np.sum(all_fold_preds, axis=0)
        votes = np.argmax(votes, axis=1)

        write_path = os.path.join(args.output_dir, "output_softvote.csv")
        #write_path = "output_softvote.csv"
        if not os.path.exists(args.output_dir):
            os.makedirs(args.output_dir)
        with open(write_path, 'w', encoding='utf8') as w:
            print("writing prediction : {}".format(write_path))
            w.write("index,topic_idx\n")
            for id, p in zip(total_ids, votes):
                w.write('{},{}\n'.format(id, p))


def train(args, model, tokenizer, train_loader, optimizer):
    model.train()

    total_preds = []
    total_targets = []
    losses = []
    for step, batch in tqdm(enumerate(train_loader), desc='Training', total=len(train_loader)):
        idx, text, label = batch
        label = label.to(args.device)
        # print(idx[:10])
        # print(text[:10])
        # print(label[:10])
        tokenized_examples = tokenizer(
            text,
            max_length=args.max_seq_len,
            padding="max_length",
            return_tensors="pt"
        ).to(args.device)
        
        # tokenize
        # 모델의 입력으로
        # label은 one-hot?
        # loss 주고
        # argmax를 golden

        token_label_0_type_ids = []
        token_label_1_type_ids = []
        token_label_2_type_ids = []
        token_label_3_type_ids = []
        token_label_4_type_ids = []
        token_label_5_type_ids = []
        token_label_6_type_ids = []

        for row in tokenized_examples['input_ids']:
            row = tokenizer.convert_ids_to_tokens(row)
            # print(row)
            label_0 = [1 if token in args.vocab[0] else 0 for token in row]
            label_1 = [1 if token in args.vocab[1] else 0 for token in row]
            label_2 = [1 if token in args.vocab[2] else 0 for token in row]
            label_3 = [1 if token in args.vocab[3] else 0 for token in row]
            label_4 = [1 if token in args.vocab[4] else 0 for token in row]
            label_5 = [1 if token in args.vocab[5] else 0 for token in row]
            label_6 = [1 if token in args.vocab[6] else 0 for token in row]
            # print(label_2)
            token_label_0_type_ids.append(label_0)
            token_label_1_type_ids.append(label_1)
            token_label_2_type_ids.append(label_2)
            token_label_3_type_ids.append(label_3)
            token_label_4_type_ids.append(label_4)
            token_label_5_type_ids.append(label_5)
            token_label_6_type_ids.append(label_6)
            # temp = []
            # row = tokenizer.convert_ids_to_tokens(row)
            # for token in row:
            #     for i, element in enumerate(args.vocab):
            #         if token in element:
            #             temp.append(i+1)
            #             break
            #     else:
            #         temp.append(0)
            # #print(temp)
            # token_label_type_ids.append(temp)

        # token_label_type_ids = torch.tensor(token_label_type_ids).to(args.device)
        # tokenized_examples['token_label_type_ids'] = token_label_type_ids

        token_label_0_type_ids = torch.tensor(token_label_0_type_ids).to(args.device)
        token_label_1_type_ids = torch.tensor(token_label_1_type_ids).to(args.device)
        token_label_2_type_ids = torch.tensor(token_label_2_type_ids).to(args.device)
        token_label_3_type_ids = torch.tensor(token_label_3_type_ids).to(args.device)
        token_label_4_type_ids = torch.tensor(token_label_4_type_ids).to(args.device)
        token_label_5_type_ids = torch.tensor(token_label_5_type_ids).to(args.device)
        token_label_6_type_ids = torch.tensor(token_label_6_type_ids).to(args.device)

        tokenized_examples['token_label_0_type_ids'] = token_label_0_type_ids
        tokenized_examples['token_label_1_type_ids'] = token_label_1_type_ids
        tokenized_examples['token_label_2_type_ids'] = token_label_2_type_ids
        tokenized_examples['token_label_3_type_ids'] = token_label_3_type_ids
        tokenized_examples['token_label_4_type_ids'] = token_label_4_type_ids
        tokenized_examples['token_label_5_type_ids'] = token_label_5_type_ids
        tokenized_examples['token_label_6_type_ids'] = token_label_6_type_ids

        preds = model(**tokenized_examples, labels = label)
        # print(preds)
        logits = preds['logits']
        # logits = logits[:,0,:]
        softmax_logits = nn.Softmax(dim=1)(logits)
        argmax_logits = torch.argmax(logits, dim=1)

        # one_hot_logits = one_hot(argmax_logits, num_classes=7).float()
        # print(one_hot(argmax_logits, num_classes=7).type(torch.FloatTensor))
        # loss = compute_loss(logits,
        #                     label, args)
        loss = preds['loss']
        # print(loss)

        update_params(loss, model, optimizer, step, len(train_loader), args)

        if step % args.log_steps == 0:
            print(f"Training steps: {step} Loss: {str(loss.item())}")

        if args.device == 'cuda':
            argmax_logits = argmax_logits.to('cpu').detach().numpy()
            label = label.to('cpu').detach().numpy()
            loss = loss.to('cpu').detach().numpy()
            # token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
            token_label_0_type_ids = token_label_0_type_ids.to('cpu').detach().numpy()
            token_label_1_type_ids = token_label_1_type_ids.to('cpu').detach().numpy()
            token_label_2_type_ids = token_label_2_type_ids.to('cpu').detach().numpy()
            token_label_3_type_ids = token_label_3_type_ids.to('cpu').detach().numpy()
            token_label_4_type_ids = token_label_4_type_ids.to('cpu').detach().numpy()
            token_label_5_type_ids = token_label_5_type_ids.to('cpu').detach().numpy()
            token_label_6_type_ids = token_label_6_type_ids.to('cpu').detach().numpy()
        else:  # cpu
            argmax_logits = argmax_logits.detach().numpy()
            label = label.detach().numpy()
            loss = loss.detach().numpy()
            # token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
            token_label_0_type_ids = token_label_0_type_ids.detach().numpy()
            token_label_1_type_ids = token_label_1_type_ids.detach().numpy()
            token_label_2_type_ids = token_label_2_type_ids.detach().numpy()
            token_label_3_type_ids = token_label_3_type_ids.detach().numpy()
            token_label_4_type_ids = token_label_4_type_ids.detach().numpy()
            token_label_5_type_ids = token_label_5_type_ids.detach().numpy()
            token_label_6_type_ids = token_label_6_type_ids.detach().numpy()

        total_preds.append(argmax_logits)
        total_targets.append(label)
        losses.append(loss)

    total_preds = np.concatenate(total_preds)
    total_targets = np.concatenate(total_targets)

    # Train AUC / ACC
    acc = accuracy_score(total_targets, total_preds)
    loss_avg = sum(losses) / len(losses)
    print(f'TRAIN ACC : {acc}, TRAIN LOSS : {loss_avg}')
    return acc, loss_avg


def validate(args, model, tokenizer, valid_loader):
    model.eval()

    total_preds = []
    total_targets = []
    losses = []
    for step, batch in tqdm(enumerate(valid_loader), desc='Training', total=len(valid_loader)):
        idx, text, label = batch
        label = label.to(args.device)
        tokenized_examples = tokenizer(
            text,
            max_length=args.max_seq_len,
            padding="max_length",
            return_tensors="pt"
        ).to(args.device)

        # tokenize
        # 모델의 입력으로
        # label은 one-hot?
        # loss 주고
        # argmax를 golden

        token_label_0_type_ids = []
        token_label_1_type_ids = []
        token_label_2_type_ids = []
        token_label_3_type_ids = []
        token_label_4_type_ids = []
        token_label_5_type_ids = []
        token_label_6_type_ids = []

        for row in tokenized_examples['input_ids']:
            row = tokenizer.convert_ids_to_tokens(row)
            # print(row)
            label_0 = [1 if token in args.vocab[0] else 0 for token in row]
            label_1 = [1 if token in args.vocab[1] else 0 for token in row]
            label_2 = [1 if token in args.vocab[2] else 0 for token in row]
            label_3 = [1 if token in args.vocab[3] else 0 for token in row]
            label_4 = [1 if token in args.vocab[4] else 0 for token in row]
            label_5 = [1 if token in args.vocab[5] else 0 for token in row]
            label_6 = [1 if token in args.vocab[6] else 0 for token in row]
            # print(label_2)
            token_label_0_type_ids.append(label_0)
            token_label_1_type_ids.append(label_1)
            token_label_2_type_ids.append(label_2)
            token_label_3_type_ids.append(label_3)
            token_label_4_type_ids.append(label_4)
            token_label_5_type_ids.append(label_5)
            token_label_6_type_ids.append(label_6)
            # temp = []
            # row = tokenizer.convert_ids_to_tokens(row)
            # for token in row:
            #     for i, element in enumerate(args.vocab):
            #         if token in element:
            #             temp.append(i+1)
            #             break
            #     else:
            #         temp.append(0)
            # #print(temp)
            # token_label_type_ids.append(temp)

        # token_label_type_ids = torch.tensor(token_label_type_ids).to(args.device)
        # tokenized_examples['token_label_type_ids'] = token_label_type_ids

        token_label_0_type_ids = torch.tensor(token_label_0_type_ids).to(args.device)
        token_label_1_type_ids = torch.tensor(token_label_1_type_ids).to(args.device)
        token_label_2_type_ids = torch.tensor(token_label_2_type_ids).to(args.device)
        token_label_3_type_ids = torch.tensor(token_label_3_type_ids).to(args.device)
        token_label_4_type_ids = torch.tensor(token_label_4_type_ids).to(args.device)
        token_label_5_type_ids = torch.tensor(token_label_5_type_ids).to(args.device)
        token_label_6_type_ids = torch.tensor(token_label_6_type_ids).to(args.device)

        tokenized_examples['token_label_0_type_ids'] = token_label_0_type_ids
        tokenized_examples['token_label_1_type_ids'] = token_label_1_type_ids
        tokenized_examples['token_label_2_type_ids'] = token_label_2_type_ids
        tokenized_examples['token_label_3_type_ids'] = token_label_3_type_ids
        tokenized_examples['token_label_4_type_ids'] = token_label_4_type_ids
        tokenized_examples['token_label_5_type_ids'] = token_label_5_type_ids
        tokenized_examples['token_label_6_type_ids'] = token_label_6_type_ids

        # token_label_type_ids = []
        # for row in tokenized_examples['input_ids']:
        #     temp = []
        #     row = tokenizer.convert_ids_to_tokens(row)
        #     for token in row:
        #         for i, element in enumerate(args.vocab):
        #             if token in element:
        #                 temp.append(i+1)
        #                 break
        #         else:
        #             temp.append(0)
        #     #print(temp)
        #     token_label_type_ids.append(temp)

        # token_label_type_ids = torch.tensor(token_label_type_ids).to(args.device)
        # tokenized_examples['token_label_type_ids'] = token_label_type_ids

        preds = model(**tokenized_examples, labels = label)
        logits = preds['logits']
        # logits = logits[:,0,:]
        softmax_logits = nn.Softmax(dim=1)(logits)
        argmax_logits = torch.argmax(logits, dim=1)

        # one_hot_logits = one_hot(argmax_logits, num_classes=7).float()
        # print(one_hot(argmax_logits, num_classes=7).type(torch.FloatTensor))
        # loss = compute_loss(logits,
        #                     label, args)
        loss = preds['loss']

        if step % args.log_steps == 0:
            print(f"Validation steps: {step} Loss: {str(loss.item())}")

        # if args.device == 'cuda':
        #     argmax_logits = argmax_logits.to('cpu').detach().numpy()
        #     label = label.to('cpu').detach().numpy()
        #     loss = loss.to('cpu').detach().numpy()
        #     token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
        # else:  # cpu
        #     argmax_logits = argmax_logits.detach().numpy()
        #     label = label.detach().numpy()
        #     loss = loss.detach().numpy()
        #     token_label_type_ids = token_label_type_ids.detach().numpy()

        if args.device == 'cuda':
            argmax_logits = argmax_logits.to('cpu').detach().numpy()
            label = label.to('cpu').detach().numpy()
            loss = loss.to('cpu').detach().numpy()
            # token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
            token_label_0_type_ids = token_label_0_type_ids.to('cpu').detach().numpy()
            token_label_1_type_ids = token_label_1_type_ids.to('cpu').detach().numpy()
            token_label_2_type_ids = token_label_2_type_ids.to('cpu').detach().numpy()
            token_label_3_type_ids = token_label_3_type_ids.to('cpu').detach().numpy()
            token_label_4_type_ids = token_label_4_type_ids.to('cpu').detach().numpy()
            token_label_5_type_ids = token_label_5_type_ids.to('cpu').detach().numpy()
            token_label_6_type_ids = token_label_6_type_ids.to('cpu').detach().numpy()
        else:  # cpu
            argmax_logits = argmax_logits.detach().numpy()
            label = label.detach().numpy()
            loss = loss.detach().numpy()
            # token_label_type_ids = token_label_type_ids.to('cpu').detach().numpy()
            token_label_0_type_ids = token_label_0_type_ids.detach().numpy()
            token_label_1_type_ids = token_label_1_type_ids.detach().numpy()
            token_label_2_type_ids = token_label_2_type_ids.detach().numpy()
            token_label_3_type_ids = token_label_3_type_ids.detach().numpy()
            token_label_4_type_ids = token_label_4_type_ids.detach().numpy()
            token_label_5_type_ids = token_label_5_type_ids.detach().numpy()
            token_label_6_type_ids = token_label_6_type_ids.detach().numpy()
        total_preds.append(argmax_logits)
        total_targets.append(label)
        losses.append(loss)

    total_preds = np.concatenate(total_preds)
    total_targets = np.concatenate(total_targets)

    # Train AUC / ACC
    target_names = ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
    print(metrics.classification_report(total_targets, total_preds, target_names=target_names))
    matrix = metrics.confusion_matrix(total_targets, total_preds)
    print(matrix.diagonal()/matrix.sum(axis=1))

    acc = accuracy_score(total_targets, total_preds)
    loss_avg = sum(losses) / len(losses)
    print(f'VALID ACC : {acc}, VALID LOSS : {loss_avg}')
    return acc, loss_avg


## Train

In [7]:
import torch
from sklearn.model_selection import KFold, StratifiedKFold
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from datetime import datetime
from pytz import timezone


def main(args):
    if not args.run_name:
        args.run_name = datetime.now(timezone("Asia/Seoul")).strftime("%Y-%m-%d-%H:%M:%S")

    set_seeds(args.seed)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    args.device = device

    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name
        if args.tokenizer_name
        else args.model_name_or_path,
        use_fast=True,
    )

    preprocess = Preprocess(args)
    preprocess.load_train_data()
    train_data_origin = preprocess.train_data

    print(f"Size of train data : {len(train_data_origin)}")
    # print(f"size of test data : {len(test_data)}")

    if args.cv_strategy == 'random':
        kf = KFold(n_splits=args.fold_num, shuffle=True)
        splits = kf.split(X=train_data_origin)
    else:
        # default
        # 여기 각 label로 바꿔야됨
        train_labels = [sequence[-1] for sequence in train_data_origin]
        skf = StratifiedKFold(n_splits=args.fold_num, shuffle=True)
        splits = skf.split(X=train_data_origin, y=train_labels)

    acc_avg = 0
    for fold_num, (train_index, valid_index) in enumerate(splits):
        train_data = train_data_origin[train_index]
        valid_data = train_data_origin[valid_index]
        best_acc = run(args, tokenizer, train_data, valid_data, fold_num + 1)

        if not args.cv_strategy:
            break

        acc_avg += best_acc

    if args.cv_strategy:
        acc_avg /= args.fold_num

        print("*" * 50, 'auc_avg', "*" * 50)
        print(acc_avg)


## Run

In [8]:
import argparse
import easydict

def parse_args():
    args = easydict.EasyDict({'run_name' : 'temp',
                             'seed':42,
                             'device' :'cuda',
                             'data_dir': cur_dir + '/data/open/',
                             'model_dir' : '/content/drive/MyDrive/KLUE_TC/models',
                             'model_name_or_path' : 'klue/roberta-large',
                             'config_name' : None,
                             'tokenizer_name' : None,
                             'output_dir' : '/content/drive/MyDrive/KLUE_TC/output/vocab20_multi_layer',
                             'vocab_dir' : '/content/drive/MyDrive/KLUE_TC/data/vocab/vocab20/',
                             
                             'accum_iter' : 8,
                             'gradient_accumulation' : True,

                             'cv_strategy' : 'stratified',
                             'fold_num' : 4,

                             'num_workers' : 1,

                             # 훈련
                             'n_epochs' : 20,
                             'batch_size' : 32,
                             'lr' : 5e-6,
                             'clip_grad' : 15,
                             'patience' : 3,
                             'max_seq_len' : 40,

                             # Optimizer
                             'optimizer' : 'adamP',

                             # Optimizer-parameters
                             'weight_decay' : 0.05,
                             'momentum' : 0.9,

                             # Scheduler
                             'scheduler' : 'step_lr',

                             # Scheduler-parameters
                             # plateau
                             'plateau_patience' : 10,
                             'plateau_factor' : 0.5,
                              
                             't_max' : 10,
                             'T_0' : 10,
                             'T_mult' : 2,
                             '--eta_min' : 0.01,

                             # linear_warmup
                             'warmup_ratio' : 0.3,

                             # Step LR
                             'step_size' : 50,
                             'gamma' : 0.1,

                             'criterion' : 'CE',

                             'log_steps' : 100})
    
    return args

In [9]:
if __name__ == '__main__':
    args = parse_args()
    args['vocab'] = make_vocab(args)
    main(args)

============ READ VOCABS ============
category 0 reading end, size : 1446
category 1 reading end, size : 1522
category 2 reading end, size : 2107
category 3 reading end, size : 1853
category 4 reading end, size : 2006
category 5 reading end, size : 1527
category 6 reading end, size : 1692



Size of train data : 45654


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'roberta.embeddings.token_label_2_

Start Training: Epoch 1


Training:   0%|          | 1/1070 [00:00<14:18,  1.25it/s]

...

Validation steps: 200 Loss: 0.5479438900947571


Training:  84%|████████▍ | 301/357 [01:32<00:17,  3.24it/s]

Validation steps: 300 Loss: 0.785088300704956


Training: 100%|██████████| 357/357 [01:49<00:00,  3.26it/s]

              precision    recall  f1-score   support

        IT과학       0.81      0.87      0.84      1206
          경제       0.81      0.86      0.83      1555
          사회       0.87      0.70      0.77      1840
        생활문화       0.88      0.92      0.90      1484
          세계       0.91      0.93      0.92      1907
         스포츠       0.95      0.98      0.96      1733
          정치       0.91      0.92      0.91      1688

    accuracy                           0.88     11413
   macro avg       0.88      0.88      0.88     11413
weighted avg       0.88      0.88      0.88     11413

[0.86981758 0.85980707 0.6951087  0.91576819 0.93340325 0.97864974
 0.91528436]
VALID ACC : 0.8801366862349952, VALID LOSS : 0.382019029909364
{'epoch': 12, 'train_loss': 0.24281824840342298, 'train_acc': 0.9154230308694256, 'valid_acc': 0.8801366862349952, 'val_loss': 0.382019029909364, 'learning_rate': 5e-06}
EarlyStopping counter: 3 out of 3
************************************************** auc_a

In [9]:
torch.cuda.empty_cache()

## Inference

In [10]:
def inference_main():
    args = parse_args()
    args['vocab'] = make_vocab(args)
    args.model_name = "temp"
    preprocess = Preprocess(args)
    preprocess.load_test_data()
    test_data = preprocess.test_data

    print(f"size of test data : {len(test_data)}")
    torch.cuda.empty_cache()
    # del model
    inference(args, test_data)

inference_main()

============ READ VOCABS ============
category 0 reading end, size : 1446
category 1 reading end, size : 1522
category 2 reading end, size : 2107
category 3 reading end, size : 1853
category 4 reading end, size : 2006
category 5 reading end, size : 1527
category 6 reading end, size : 1692
size of test data : 9131
Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_1.pt


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.embeddings.token_label_0_type_embeddings.weight', 'c

Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_1.pt ...Finished.


Inferencing: 100%|██████████| 286/286 [01:41<00:00,  2.81it/s]


writing prediction : /content/drive/MyDrive/KLUE_TC/output/vocab20_multi_layer/output_1.csv
Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_2.pt


...

writing prediction : /content/drive/MyDrive/KLUE_TC/output/vocab20_multi_layer/output_3.csv
Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_4.pt


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.embeddings.token_label_0_type_embeddings.weight', 'c

Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_4.pt ...Finished.


Inferencing: 100%|██████████| 286/286 [00:30<00:00,  9.34it/s]


writing prediction : /content/drive/MyDrive/KLUE_TC/output/vocab20_multi_layer/output_4.csv
writing prediction : /content/drive/MyDrive/KLUE_TC/output/vocab20_multi_layer/output_softvote.csv
